In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain_core.documents import Document
from utils.load_json import get_chunk_with_json




json_file = "/Users/admin/Working/nckh-chatbot-neo4j/crawl_data/fit.jsonl"
webs = get_chunk_with_json(json_file)
# loader = JSONLoader(
#     file_path=json_file,
#     jq_schema='.id',
#     text_content=False,
#     json_lines=True
# )
docs = []
for data in webs:
    page_content = data['kwargs']['page_content']
        
    title = data['kwargs']['metadata']['title']
    source = data['kwargs']['metadata']['source']
    if len(page_content)>500:
        doc = Document(page_content=page_content, metadata={"source": source, "title":title})
        docs.append(doc)


In [2]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from llm.get_llm import generate_test_model_function, get_embedding_function, get_model_function
generator_llm = LangchainLLMWrapper(get_model_function())
generator_embeddings = LangchainEmbeddingsWrapper(get_embedding_function())

from ragas.testset.persona import Persona

personas = [
    Persona(
        name="sinh viên tò mò",
        role_description="Một sinh viên Việt Nam đang tiến hành nghiên cứu về các nội dung liên quan đến hoạt động của Khoa công nghệ thông tin Trường Đại học Công nghiệp Hà Nội. Các nội dung liện quan tới các ngành, các hoạt động, các cuộc thi, các câu lạc bộ. Câu hỏi sẽ ở ngôn ngữ tiếng Việt",
    ),
    Persona(
        name="Giảng viên tận tâm",
        role_description="Một giảng viên tại Khoa Công nghệ Thông tin Trường Đại học Công nghiệp Hà Nội, luôn tìm kiếm cách nâng cao chất lượng giảng dạy và hỗ trợ sinh viên. Quan tâm đến các hoạt động học thuật, đổi mới phương pháp giảng dạy, và phát triển các chương trình hợp tác với doanh nghiệp. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),
    Persona(
        name="Học sinh đang tìm hiểu",
        role_description="Một học sinh lớp 12 tại Việt Nam đang cân nhắc đăng ký vào khoa công nghệ thông tin Đại học Công nghiệp Hà Nội, đặc biệt quan tâm đến ngành Công nghệ Thông tin. Mong muốn tìm hiểu về chương trình đào tạo, cơ hội nghề nghiệp sau khi tốt nghiệp, và các tiêu chí xét tuyển của trường. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),
    Persona(
        name="Phụ huynh lo lắng",
        role_description="Một phụ huynh có con đang chuẩn bị xét tuyển vào khoa công nghệ thông tin  Đại học Công nghiệp Hà Nội. Quan tâm đến môi trường học tập, chất lượng đào tạo, cơ hội việc làm sau khi tốt nghiệp, cũng như chi phí học tập và hỗ trợ sinh viên tại trường. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),


]

/opt/anaconda3/envs/test_ragas3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅✅Environment file in llm loaded successfully
get llm 
get_embedding_function


In [3]:
from ragas.testset import TestsetGenerator
from ragas.cost import get_token_usage_for_openai

generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
]

# for query, _ in distribution:
    # prompts = await query.adapt_prompts("spanish", llm=generator_llm)
    # query.set_prompts(**prompts)
    # query.save_prompts("/Users/admin/Working/thaibinh-chatbot/evaluate")

In [6]:
from ragas.cost import get_token_usage_for_openai

dataset = generator.generate_with_langchain_docs(
    docs[50:140],
    testset_size=90,
    query_distribution=distribution,
    
)

Applying HeadlineSplitter:   0%|          | 0/90 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
Applying Sum

In [5]:
import datetime

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the current date and time
formatted_datetime = current_datetime.strftime("%y%m%d-%H%M")

# Create a dynamic filename
filename = f"test_fit{formatted_datetime}"
eval_dataset = dataset.to_evaluation_dataset()
eval_dataset.to_csv(f"{filename}.csv")